# Notkun gervigreindar fyrir teikningu þrívíddarmynda

Nathan HK

In [2]:
import numpy as np
import os
import pywikibot
import sklearn as sk
from sklearn.model_selection import train_test_split
from stl import mesh
import time
import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
torch.set_default_device(device)
mappa = '/Users/002-nathan/Desktop/Envalys/envalys-nathan/'  # Change this

## Inngangsorð
Við ætlum að þjálfa gervigreindarlíkan til að teikna þrívíddarmyndir. Hvernig gerum við það?
- Fyrst reyndi ég að nota þrívíddardíla eða "þríla" (e. *voxels*). Ég taldi punktana í hverjum þríl, og þjálfaði líkan með tölunum, eins og líkan sem greinir ljósmyndir. GitHub: https://github.com/nholmesking/envalys-nathan/blob/4fd89a1515c1c3623a5b8cdf8c766858c465e451/Nathan%20HK.ipynb
  - Vandamál #1: Mac-GPU getur ekki neitt með þrívíddargögnum. Ég þurfti að nota CPU, sem er of hægt.
  - Vandamál #2: Hraðinn er $O(n^3)$.
  - Vandamál #3: Það var engin klár leið til að fara frá greiningu yfir á teikningu.
- Þá reyndi ég að nota líkanið 3DShape2VecSet, sem tekur lista yfir punkta og teinkir myndir sem eru eins og listinn.

Tilvísanir:
- https://arxiv.org/pdf/2301.11445
- https://github.com/1zb/3DShape2VecSet

Ég gerði litlar breytingar á upprunaforritið, því það er eitt forritasafn (e. *module*) sem virkar ekki lengur, og forritið var skrifað fyrir Windows (og ég nota Mac).

## Gögn
Gögnin frá 3DShape2VecSet eru 650 GB samtals; þetta er of mikið fyrir tölvuna mína.

Þessi gögn eru STL-skrár frá Wikimedia Commons. Það eru fimm flokkar:
- líkamshlutar
- byggingar
- rúmfræði
- geimfarartæki
- styttur

### Sækja gögn

In [3]:
flokkar = ['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
skrar = {}
catnum = {}

In [4]:
commons = pywikibot.Site('commons', 'commons')
cn = 0
for a in flokkar:
    print(a)
    cat = pywikibot.Category(commons, 'STL files of ' + a)
    catnum[a] = cn
    cn += 1
    n = 0
    for p in cat.members(member_type=['file']):
        if n % 10 == 0:
            print(n)
        mynd = pywikibot.FilePage(p)
        try:
            tempf = open(mappa + 'STLdata/' + a + '_' + p.title()[5:], 'r')
            tempf.close()
        except FileNotFoundError:
            mynd.download(filename=mappa + 'STLdata/' + a + '_' + p.title()[5:])
        try:
            skrar[a].append(p.title()[5:])
        except KeyError:
            skrar[a] = [p.title()[5:]]
        n += 1
        if n >= 100:
            break

body parts
0
10
20
30
40
50
60
70
80
buildings
0
10
20
geometric shapes
0
10
20
30
40
objects in space
0
10
20
30
40
50
sculptures
0
10
20
30
40
50


### Setja upp gögn fyrir notkun
Við eigum að breyta gögnunum úr STL-sniði í sniðið sem 3DShape2VecSet notar. Þetta snið er með lista yfir punkta eftir hvort þeir séu innan á forminu eða utan, en gögnin frá Wikimedia Commons er ekki með það.

Með ```isInside()``` getum útreiknað hvort lína og þríhyrningur mætast, og í hvora átt. Við búum til tvær línur sem fara beint í X-, Y-, eða Z-áttina frá punktnum (eina línu upp og eina línu niður), og teljum þríhyrningana sem línan mætir; ef talan er slétt, þá er punkturinn utan á forminu, og ef talan er oddatala, þá er punkturinn innan. En það tekur of langan tíma; sumar myndir eru með fleiri en milljón þríhyrninga. 

Þess vegna bjó ég til þetta reknirit: Við deilum myndinni í 32x32 eða fleiri ílát í þrem ílátahópum, einum fyrir hverja vídd, eftir hnitum hinna vídda. Í byrjun setjum við alla þríhyrninga í þeirra ílát; sumir þríhyrningar, sem eru á línunni milli íláta, eru settir í tvö eða fleiri ílát. Fyrir hvern listapunkt eigum við bara að leita í rétta ílátinu, í hópnum þar sem þríhyrningafjöldinn er lægstur.

Með þessu reikniriti er sniðbreytingin nærri **400** sinnum fljótari. Ég skrifaði sjálfur allt hérna nema ```isInside()```; sjáðu kóðann fyrir tilvísun.

In [14]:
def isInside(a, randpoint, att):
    if (a[0] > randpoint[0] and a[3] > randpoint[0] and a[6] > randpoint[0]):
        return 0
    if (a[0] < randpoint[0] and a[3] < randpoint[0] and a[6] < randpoint[0]):
        return 0
    if (a[1] > randpoint[1] and a[4] > randpoint[1] and a[7] > randpoint[1]):
        return 0
    if (a[1] < randpoint[1] and a[4] < randpoint[1] and a[7] < randpoint[1]):
        return 0
 
    # This part is from https://diegoinacio.github.io/creative-coding-notebooks-page/pages/ray-intersection_triangle.html
    if att == 'x':
        e = np.array([1, 0, 0])     # Ray direction
    elif att == 'y':
        e = np.array([0, 1, 0])     # Ray direction
    elif att == 'z':
        e = np.array([0, 0, 1])     # Ray direction
    AB = a[3:6] - a[0:3]               # Oriented segment A to B
    AC = a[6:9] - a[0:3]               # Oriented segment A to C
    n = np.cross(AB, AC)     # Normal vector
    n_ = n/np.linalg.norm(n) # Normalized normal
    if not np.any(np.dot(n_, e)):
        return 0
    d = - np.dot(n_, a[0:3])
    # Finding parameter t
    t = - (np.dot(n_, randpoint) + d)/np.dot(n_, e)
    P = randpoint + t*e
    # Get the resulting vector for each vertex
    # following the construction order
    Pa = np.dot(np.cross(a[3:6] - a[0:3], P - a[0:3]), n_)
    Pb = np.dot(np.cross(a[6:9] - a[3:6], P - a[3:6]), n_)
    Pc = np.dot(np.cross(a[0:3] - a[6:9], P - a[6:9]), n_)

    if t > 0 and (Pa > 0 and Pb > 0 and Pc > 0):
        return 1
        
    if att == 'x':
        e = np.array([-1, 0, 0])     # Ray direction
    elif att == 'y':
        e = np.array([0, -1, 0])     # Ray direction
    elif att == 'z':
        e = np.array([0, 0, -1])     # Ray direction
    # Finding parameter t
    t = - (np.dot(n_, randpoint) + d)/np.dot(n_, e)
    P = randpoint + t*e
    # Get the resulting vector for each vertex
    # following the construction order
    Pa = np.dot(np.cross(a[3:6] - a[0:3], P - a[0:3]), n_)
    Pb = np.dot(np.cross(a[6:9] - a[3:6], P - a[3:6]), n_)
    Pc = np.dot(np.cross(a[0:3] - a[6:9], P - a[6:9]), n_)

    if t > 0 and (Pa > 0 and Pb > 0 and Pc > 0):
        return -1
    return 0

In [19]:
for cat in skrar:
    tts = []
    print(cat)
    byrjun = time.time()
    os.system('mkdir "' + mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '"')
    os.system('mkdir "' + mappa + '3DShape2VecSet/data/ShapeNetV2_watertight/' + cat + '"')
    for fi in skrar[cat]:
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        utskra_mesh = open(mappa + '3DShape2VecSet/data/ShapeNetV2_watertight/' + cat + '/' + fi + '.npz', 'wb')
        np.savez(utskra_mesh, points=np.array(gogn.v0, dtype=np.float32), scale=1)
        tts.append(fi)
    train, val = train_test_split(tts)
    trainlst = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/train.lst', 'w')
    print('train')
    for fi in train:
        print(fi)
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        print(gogn.points.shape[0])
        mm = ((min([a[0] for a in gogn.v0]), max([a[0] for a in gogn.v0])),
              (min([a[1] for a in gogn.v0]), max([a[1] for a in gogn.v0])),
              (min([a[2] for a in gogn.v0]), max([a[2] for a in gogn.v0])))
        num_ilat = max(32, int((gogn.points.shape[0] / 500) ** (1/2)))
        ilat_X = []
        for i in range(num_ilat):
            ilat_X.append([])
            for j in range(num_ilat):
                ilat_X[i].append([])
        ilat_Y = []
        for i in range(num_ilat):
            ilat_Y.append([])
            for j in range(num_ilat):
                ilat_Y[i].append([])
        ilat_Z = []
        for i in range(num_ilat):
            ilat_Z.append([])
            for j in range(num_ilat):
                ilat_Z[i].append([])
        ilatid = lambda x, y, z: (max(0, min(int((x - mm[0][0]) * num_ilat / (mm[0][1] - mm[0][0])), num_ilat - 1)),
                                  max(0, min(int((y - mm[1][0]) * num_ilat / (mm[1][1] - mm[1][0])), num_ilat - 1)),
                                  max(0, min(int((z - mm[2][0]) * num_ilat / (mm[2][1] - mm[2][0])), num_ilat - 1)))
        utskra = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/' + fi + '.npz', 'wb')
        vol_points = []
        vol_label = []
        for a in gogn.points:
            minid = ilatid(min([a[0], a[3], a[6]]), min([a[1], a[4], a[7]]), min([a[2], a[5], a[8]]))
            maxid = ilatid(max([a[0], a[3], a[6]]), max([a[1], a[4], a[7]]), max([a[2], a[5], a[8]]))
            for i in range(minid[0], maxid[0] + 1):
                for j in range(minid[1], maxid[1] + 1):
                    ilat_Z[i][j].append(a)
            for i in range(minid[0], maxid[0] + 1):
                for k in range(minid[2], maxid[2] + 1):
                    ilat_Y[i][k].append(a)
            for j in range(minid[1], maxid[1] + 1):
                for k in range(minid[2], maxid[2] + 1):
                    ilat_X[j][k].append(a)
        for i in range(4096):
            randpoint = (np.random.uniform(mm[0][0], mm[0][1]),
                         np.random.uniform(mm[1][0], mm[1][1]),
                         np.random.uniform(mm[2][0], mm[2][1]))
            # Is point inside or outside shape?
            fata = ilatid(randpoint[0], randpoint[1], randpoint[2])
            pct = [0, 0]
            if len(ilat_X[fata[1]][fata[2]]) <= len(ilat_Y[fata[0]][fata[2]]) and \
               len(ilat_X[fata[1]][fata[2]]) <= len(ilat_Z[fata[0]][fata[1]]):
                for a in ilat_X[fata[1]][fata[2]]:
                    isi = isInside(a, randpoint, 'x')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            elif len(ilat_Y[fata[0]][fata[2]]) <= len(ilat_X[fata[1]][fata[2]]) and \
                 len(ilat_Y[fata[0]][fata[2]]) <= len(ilat_Z[fata[0]][fata[1]]):
                for a in ilat_Y[fata[0]][fata[2]]:
                    isi = isInside(a, randpoint, 'y')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            elif len(ilat_Z[fata[0]][fata[1]]) <= len(ilat_X[fata[1]][fata[2]]) and \
                 len(ilat_Z[fata[0]][fata[1]]) <= len(ilat_Y[fata[0]][fata[2]]):
                for a in ilat_Y[fata[0]][fata[2]]:
                    isi = isInside(a, randpoint, 'z')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                vol_points.append(randpoint)
                vol_label.append(pct[0] % 2)
        print('train vol', time.time() - byrjun)
        near_points = []
        near_label = []
        for i in range(4096):
            gp = gogn.points[np.random.randint(0, gogn.points.shape[0])]
            randpoint = (gp[0] + np.random.uniform(mm[0][0] / 100, mm[0][1] / 100),
                         gp[1] + np.random.uniform(mm[1][0] / 100, mm[1][1] / 100),
                         gp[2] + np.random.uniform(mm[2][0] / 100, mm[2][1] / 100))
            # Is point inside or outside shape?
            fata = ilatid(randpoint[0], randpoint[1], randpoint[2])
            pct = [0, 0]
            if len(ilat_X[fata[1]][fata[2]]) < len(ilat_Y[fata[0]][fata[2]]) and \
               len(ilat_X[fata[1]][fata[2]]) < len(ilat_Z[fata[0]][fata[1]]):
                for a in ilat_X[fata[1]][fata[2]]:
                    isi = isInside(a, randpoint, 'x')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            elif len(ilat_Y[fata[0]][fata[2]]) < len(ilat_X[fata[1]][fata[2]]) and \
                 len(ilat_Y[fata[0]][fata[2]]) < len(ilat_Z[fata[0]][fata[1]]):
                for a in ilat_Y[fata[0]][fata[2]]:
                    isi = isInside(a, randpoint, 'y')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            elif len(ilat_Z[fata[0]][fata[1]]) < len(ilat_X[fata[1]][fata[2]]) and \
                 len(ilat_Z[fata[0]][fata[1]]) < len(ilat_Y[fata[0]][fata[2]]):
                for a in ilat_Y[fata[0]][fata[2]]:
                    isi = isInside(a, randpoint, 'z')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                near_points.append(randpoint)
                near_label.append(pct[0] % 2)
        print('train near', time.time() - byrjun)
        if len(vol_points) > 0 and len(near_points) > 0:
            trainlst.write(fi + '.npz\n')
            np.savez(utskra,
                     vol_points=np.array(vol_points, dtype=np.float32),
                     vol_label=np.array(vol_label),
                     near_points=np.array(near_points, dtype=np.float32),
                     near_label=np.array(near_label))
    trainlst.close()
    vallst = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/val.lst', 'w')
    print('val')
    for fi in val:
        print(fi)
        vallst.write(fi + '.npz\n')
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        print(gogn.points.shape[0])
        mm = ((min([a[0] for a in gogn.v0]), max([a[0] for a in gogn.v0])),
              (min([a[1] for a in gogn.v0]), max([a[1] for a in gogn.v0])),
              (min([a[2] for a in gogn.v0]), max([a[2] for a in gogn.v0])))
        num_ilat = max(32, int((gogn.points.shape[0] / 500) ** (1/2)))
        ilat_X = []
        for i in range(num_ilat):
            ilat_X.append([])
            for j in range(num_ilat):
                ilat_X[i].append([])
        ilat_Y = []
        for i in range(num_ilat):
            ilat_Y.append([])
            for j in range(num_ilat):
                ilat_Y[i].append([])
        ilat_Z = []
        for i in range(num_ilat):
            ilat_Z.append([])
            for j in range(num_ilat):
                ilat_Z[i].append([])
        ilatid = lambda x, y, z: (max(0, min(int((x - mm[0][0]) * num_ilat / (mm[0][1] - mm[0][0])), num_ilat - 1)),
                                  max(0, min(int((y - mm[1][0]) * num_ilat / (mm[1][1] - mm[1][0])), num_ilat - 1)),
                                  max(0, min(int((z - mm[2][0]) * num_ilat / (mm[2][1] - mm[2][0])), num_ilat - 1)))
        utskra = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/' + fi + '.npz', 'wb')
        vol_points = []
        vol_label = []
        for a in gogn.points:
            minid = ilatid(min([a[0], a[3], a[6]]), min([a[1], a[4], a[7]]), min([a[2], a[5], a[8]]))
            maxid = ilatid(max([a[0], a[3], a[6]]), max([a[1], a[4], a[7]]), max([a[2], a[5], a[8]]))
            for i in range(minid[0], maxid[0] + 1):
                for j in range(minid[1], maxid[1] + 1):
                    ilat_Z[i][j].append(a)
            for i in range(minid[0], maxid[0] + 1):
                for k in range(minid[2], maxid[2] + 1):
                    ilat_Y[i][k].append(a)
            for j in range(minid[1], maxid[1] + 1):
                for k in range(minid[2], maxid[2] + 1):
                    ilat_X[j][k].append(a)
        for i in range(4096):
            randpoint = (np.random.uniform(mm[0][0], mm[0][1]),
                         np.random.uniform(mm[1][0], mm[1][1]),
                         np.random.uniform(mm[2][0], mm[2][1]))
            # Is point inside or outside shape?
            fata = ilatid(randpoint[0], randpoint[1], randpoint[2])
            pct = [0, 0]
            if len(ilat_X[fata[1]][fata[2]]) <= len(ilat_Y[fata[0]][fata[2]]) and \
               len(ilat_X[fata[1]][fata[2]]) <= len(ilat_Z[fata[0]][fata[1]]):
                for a in ilat_X[fata[1]][fata[2]]:
                    isi = isInside(a, randpoint, 'x')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            elif len(ilat_Y[fata[0]][fata[2]]) <= len(ilat_X[fata[1]][fata[2]]) and \
                 len(ilat_Y[fata[0]][fata[2]]) <= len(ilat_Z[fata[0]][fata[1]]):
                for a in ilat_Y[fata[0]][fata[2]]:
                    isi = isInside(a, randpoint, 'y')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            elif len(ilat_Z[fata[0]][fata[1]]) <= len(ilat_X[fata[1]][fata[2]]) and \
                 len(ilat_Z[fata[0]][fata[1]]) <= len(ilat_Y[fata[0]][fata[2]]):
                for a in ilat_Y[fata[0]][fata[2]]:
                    isi = isInside(a, randpoint, 'z')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                vol_points.append(randpoint)
                vol_label.append(pct[0] % 2)
        if len(vol_points) > 0:
            print('val vol', time.time() - byrjun)
            np.savez(utskra,
                     vol_points=np.array(vol_points, dtype=np.float32),
                     vol_label=np.array(vol_label))
    vallst.close()
    print(time.time() - byrjun)
    print('----')

body parts


mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_point/body parts: File exists
mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_watertight/body parts: File exists


train
We Are Beautiful – 507391-OSNN-1 – surface.stl
107777
train vol 4.045544862747192
train near 5.134532928466797
We Are Beautiful – 662767-FSAN-1 – solid.stl
487252
train vol 14.642663955688477
train near 17.644624948501587
Vh-m-eye-r.stl
346630
train vol 27.30580472946167
train near 48.67331576347351
We Are Beautiful – 426060-FSAN-1 – surface.stl
244908
train vol 53.986488819122314
train near 55.4530508518219
BodyParts3D FJ6400 Manubrium.stl
2526
train vol 55.88746094703674
train near 56.1829137802124
We Are Beautiful – 320912-FSAN-1 – solid.stl
597660
train vol 67.32360887527466
train near 70.2424430847168
Vh-m-main-bronchus.stl
6023
train vol 70.62866282463074
train near 72.58347988128662
Vh-m-larynx.stl
13238
train vol 73.2411139011383
train near 74.57456111907959
Reconstrução facial - busto - Cáceres-MT.stl
199966
train vol 78.57796096801758
train near 80.63899183273315
We Are Beautiful – 453703-OSNN-1 – solid.stl
98686
train vol 84.5767240524292
train near 86.8406457901001
We

  n_ = n/np.linalg.norm(n) # Normalized normal



train near 126.16487193107605
Hollow face illusion.stl
2518
train vol 126.39959669113159
train near 127.46222186088562
We Are Beautiful – 554268-FSAN-1 – solid.stl
423450
train vol 135.78525185585022
train near 138.37888288497925
Vh-m-lung.stl
224765
train vol 143.19439387321472
train near 147.98513007164001
We Are Beautiful – 591522-FSAN-2 – solid.stl
386810
train vol 155.65350890159607
train near 158.69103384017944
Vh-m-blood-vasculature.stl
416538
train vol 165.1075677871704
train near 173.60479283332825
First Rib (Human).stl
11232
train vol 173.8898298740387
train near 173.9851598739624
BodyParts3D Patella.stl
614
train vol 174.2630889415741
train near 174.88270092010498
Vh-m-heart.stl
164119
train vol 178.9091658592224
train near 182.44727993011475
Vh-m-eye-l.stl
346630
train vol 191.7717788219452
train near 213.0326919555664
We Are Beautiful – 591522-FSAN-4 – surface.stl
398721
train vol 221.5374858379364
train near 223.81683993339539
We Are Beautiful – 591522-FSAN-3 – surface.st

...

train
Fcc bz 3.stl
291
train vol 2.2193949222564697
train near 3.096625804901123
Supershape 20230523 001 Spherical Harmonics 100x100.stl
8000000
train vol 99.52704191207886
train near 188.53809690475464
LowPoly Sphere 05.stl
26
train vol 189.7447109222412
train near 190.3411979675293
3D model of a Stellated Octahedron.stl
8
train vol 191.05603504180908
train near 191.1743938922882
Earth.stl
122880
train vol 194.1640477180481
train near 195.85205507278442
Cube.stl
12
train vol 195.98740983009338
train near 196.04964995384216
Moon elevation.stl
525694
train vol 206.9358148574829
train near 210.0466170310974
LowPoly Sphere 09.stl
86
train vol 210.58952283859253
train near 211.26516604423523
LowPoly Sphere 08.stl
60
train vol 211.8622658252716
train near 212.5754837989807
Bcc bz 2.stl
77
train vol 213.74863576889038
train near 214.56022095680237
Bcc bz 3.stl
68
train vol 215.35746574401855
train near 215.92576098442078
Soccer ball.stl
16380
train vol 216.8742368221283
train near 219.034091

mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_point/objects in space: File exists
mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_watertight/objects in space: File exists


exception (False, 'No lines found, impossible to read')
train
SAOCOM HORIZONTAL calado 1 sin circulos REP.stl
1362060
train vol 21.18002200126648
train near 46.557841777801514
Curiosity Rover.stl
114899
train vol 49.27324199676514
train near 53.06392288208008
Cassini 3D Model.stl
25736
train vol 54.043256998062134
train near 57.63706111907959
New Horizons.stl
160886
train vol 61.68175196647644
train near 68.69660377502441
Aura 27.stl
87333
train vol 71.85437202453613
train near 80.74729084968567
Dawn 3D Model.stl
32084
train vol 81.37725281715393
train near 85.39901375770569
Apollo command module smithsonian high detail.stl
3074098
train vol 127.46771883964539
train near 139.14169001579285
Aura 2013.stl
115781
train vol 142.0230278968811
train near 147.49189615249634
ICESat-2 Model 2013.stl
667494
train vol 157.50745487213135
train near 197.49796104431152
Aquarius UD.stl
10362
train vol 199.0733199119568
train near 201.72095680236816
CYGNSS solo 39.stl
7338
train vol 202.3492648601532


mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_point/sculptures: File exists
mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_watertight/sculptures: File exists


train
五神名地神塔（横浜市旭区）.stl
1473158
train vol 30.20372223854065
train near 34.04617094993591
庚申塔（寺山杉山神社）.stl
1200000
train vol 51.45736002922058
train near 54.37772297859192
出羽三山塔（菅田杉山社）.stl
1770190
train vol 80.8840811252594
train near 85.57617807388306
馬頭観音（ごはん塚）.stl
1500000
train vol 107.42444801330566
train near 110.1559488773346
Ana-Hachimangu water basin of Hotei 2024-04-25.stl
280209
train vol 115.71983599662781
train near 118.50998497009277
Wasserbüffel und Knabe - 3D model by noe-3d.at - Sketchfab.stl
661492
train vol 130.77604007720947
train near 141.401762008667
Castlestrange Stone - 3D model by roscommon3d - Sketchfab.stl
67652
train vol 143.3599350452423
train near 145.2602081298828
庚申塔道標（横浜市旭区今宿南町）.stl
1750000
train vol 168.7471330165863
train near 171.66297698020935
Bust of Emperor Caracalla.stl
181169
train vol 175.55027294158936
train near 177.5142149925232
Scan the World - Pietà (Michelangelo).stl
815738
train vol 191.84247303009033
train near 196.16832399368286
刻像地神塔（正覚寺

Sniðbreytingin tók 50 mínútur.

## Líkan
Líkanið þarf ekki að búa til góðar myndir. Það þarf bara að virka án villna, sem sýnir okkur að við þurfum bara meiri og betri gögn, því vísindagreinin er með góðar myndir.

In [18]:
os.system('cd ' + mappa)
os.system('torchrun --nproc_per_node=4 3DShape2VecSet/main_ae.py --accum_iter=2 --model ae_d512_m512 '
          '--data_path 3DShape2VecSet/data --output_dir output/ae/ae_d512_m512 --log_dir output/ae/ae_d512_m512 '
          '--num_workers 10 --point_cloud_size 2048 --batch_size 64 --epochs 200 --warmup_epochs 5')

W0825 09:05:21.680789 8286703424 torch/distributed/elastic/multiprocessing/redirects.py:28] NOTE: Redirects are currently not supported in Windows or MacOs.
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] 
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] *****************************************
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] *****************************************


['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']

['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
Model = AutoEncoder(
  (cross_attend_blocks): ModuleList(
    (0): PreNorm(
      (fn): Attention(
        (to_q): Linear(in_features=512, out_features=512, bias=False)
        (to_kv): Linear(in_features=512, out_features=1024, bias=False)
        (to_out): Linear(in_features=512, out_features=512, bias=True)
        (drop_path): Identity()


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Epoch: [0]  [ 0/47]  eta: 0:43:45  lr: 0.000000  loss: 0.7250 (0.7250)  loss_vol: 0.6592 (0.6592)  loss_near: 0.6580 (0.6580)  iou: 0.0139 (0.0139)  time: 55.8586  data: 19.8939
Epoch: [0]  [ 0/47]  eta: 0:49:22  lr: 0.000000  loss: 0.7185 (0.7185)  loss_vol: 0.6532 (0.6532)  loss_near: 0.6534 (0.6534)  iou: 0.0089 (0.0089)  time: 63.0310  data: 19.9302
Epoch: [0]  [ 0/47]  eta: 0:50:54  lr: 0.000000  loss: 0.7204 (0.7204)  loss_vol: 0.6549 (0.6549)  loss_near: 0.6550 (0.6550)  iou: 0.0234 (0.0234)  time: 64.9863  data: 16.6555
Epoch: [0]  [ 0/47]  eta: 1:23:32  lr: 0.000000  loss: 0.7272 (0.7272)  loss_vol: 0.6611 (0.6611)  loss_near: 0.6610 (0.6610)  iou: 0.0238 (0.0238)  time: 106.6386  data: 16.7561


W0825 09:35:12.375863 8286703424 torch/distributed/elastic/agent/server/api.py:688] Received Signals.SIGINT death signal, shutting down workers
W0825 09:35:12.392325 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40348 closing signal SIGINT
W0825 09:35:12.393383 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40349 closing signal SIGINT
W0825 09:35:12.393764 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40350 closing signal SIGINT
W0825 09:35:12.398679 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40351 closing signal SIGINT
W0825 09:35:42.408644 8286703424 torch/distributed/elastic/multiprocessing/api.py:875] Unable to shutdown process 40348 via Signals.SIGINT, forcefully exiting via Signals.SIGKILL
W0825 09:35:49.303166 8286703424 torch/distributed/elastic/multiprocessing/api.py:875] Unable to shutdown process 40349 via Signals.SIGINT, forcefully ex

256

Það eru 200 lotna (e. *epochs*), og hver lota tekur klukkustund; við eigum ekki nógan tíma fyrir alla þjálfunina. **En hún er hafin án villna.**

# Lokaorð
Hvert förum við héðan? 3DShape2VecSet er með margar takmarkanir.
- Það kann bara að búa til einfaldar myndir, eins og "stóll". Með greiningarlíkani getum við mögulega búið til fjölfaldar myndir. Ef notandi segir "stóll með kodda", þá býr líkanið okkar til mynd af stóli og mynd af kodda, sem líkanið sameinar.
- Það skilur ekki mannamál.
- Það er ekki með mikil gögn. Með mínu forriti hérna getum við búið til eins mikil gögn og við viljum; við getum tekið myndir frá notendum og búum til lista yfir punkta, og notað þá til að þjálfa líkanið.

Ég er viss um að, með meiri gögn og betri tölvu, við getum að minnsta kosti eitt af þessum.